In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time, pandas as pd

# 설정
MAX_POSTS = 30
BASE_SEARCH_URL = "https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword="

# 드라이버 설정
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# 사용자 입력
category = input("구분 업종: ")
keyword = input("어떤 키워드로 검색?: ")
search_url = BASE_SEARCH_URL + keyword
driver.get(search_url)
time.sleep(3)

# 게시물 링크 수집
post_links = []
cards = driver.find_elements(By.CSS_SELECTOR, "div.list_search_post a.desc_inner")
for card in cards:
    href = card.get_attribute("href")
    if href and href not in post_links:
        post_links.append(href)
    if len(post_links) >= MAX_POSTS:
        break

print(f"수집된 블로그 링크 수: {len(post_links)}")

# 게시물 상세 정보 수집
results = []

for url in post_links:
    driver.get(url)
    time.sleep(2)

    try:
        desc = driver.find_element(By.XPATH, '//meta[@property="og:description"]').get_attribute("content")
    except:
        desc = ""

    try:
        tag_elems = driver.find_elements(By.CSS_SELECTOR, "span.ell")
        hashtags = [t.text for t in tag_elems if t.text.startswith("#")]
    except:
        hashtags = []

    results.append({
        "채널/기법": "n_blog",
        "구분업종": category,
        "검색 키워드": keyword,
        "게시물 링크": url,
        "게시물 텍스트": desc,
        "해시태그 리스트": " ".join(hashtags)
    })

수집된 블로그 링크 수: 7


In [ ]:
from datetime import datetime

# 현재 시간 문자열 (2025-07-29_09-30-15)
now_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# 파일명 생성
filename = f"n_blog_crawling_result_{now_str}.csv"

# 저장
df = pd.DataFrame(results)
df.to_csv(filename, index=False, encoding="utf-8-sig")
print(f"CSV 저장 완료: {filename}")